In [ ]:
import numpy as np
import itertools
import os
import ambig_utils as au
import importlib
import inversion_utils as iu
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import clear_output

In [ ]:
importlib.reload(au)
importlib.reload(au)
clear_output()

In [ ]:
ambig_executable_path = '/mn/stornext/u3/avijeetp/codes/ambiguity_resolution/ME0-2.1.1/'
par_file = 'ambig_par'
save_dir = 'temp_ambig'
ambig_input = os.path.join(save_dir, 'ambig_input.dat')
params = au.read_ambig_par_file(par_file)
params['filename'] = ambig_input
params

In [ ]:
# au.write_ambig_par_file('test_ambig_par', params,pprint=True)

In [ ]:
au.get_par_range('tfac0', verbose=True)

In [ ]:
# for key in list(params.keys())[1:]:
#     print(key, au.get_par_range(key, verbose=False))

In [ ]:
par_key = 'tfac0'
param_ranges = {
    par_key: au.get_par_range(par_key, verbose=False, start=1, end=3, step=0.5),
    # 'bthresh': au.get_par_range('bthresh', start=500, end=700, step=50, verbose=False),
}
param_ranges

In [ ]:
# Create a list of param dictionaries by varying the parameters
params_to_vary = param_ranges.keys()
params_list = []
id = 0
id_dict = {}
for param in params_to_vary:
    param_range = param_ranges[param]
    # print(f'{param}: {param_range}')
    # create a copy of the existing params dictionary
    new_params = params.copy()
    for value in param_range:
        new_params[param] = value
        params_list.append(new_params.copy())
        print(f'{id}: [{param}, {value}]')
        id_dict[id] = [param, value]
        id += 1
        # print(new_params)

In [ ]:
# restore variables from 'temp_ambig_par/ambig_test.npz'
npzpath = os.path.join(save_dir, 'ambig_input.npz')
npzfile = np.load(npzpath)

In [ ]:
count = 0
par_files = []
for new_param in params_list:
    par_file = f'ambig_par_{count}'
    par_file = os.path.join(save_dir, par_file)
    au.write_ambig_par_file(par_file, new_param,pprint=False)
    count += 1
    par_files.append(par_file)

In [ ]:
# Print the keys of the npz file
print(npzfile.files)

In [ ]:
pix = npzfile['pix'].item()
pbr = npzfile['pbr'][0], npzfile['pbr'][1], npzfile['pbr'][2]
lonlat = npzfile['lonlat'][0], npzfile['lonlat'][1]
blos = npzfile['blos']
bhor = npzfile['bhor']
bazi = npzfile['bazi']

In [ ]:
au.write_ambig_input(save_dir, pix, pbr, lonlat, blos, bhor, bazi)

In [ ]:
ambig_outfiles = []
ambig_metrics = []
ambig_figs = []
data_mask =  blos != 0 # create a mask for avoiding regions where blos=0
for id in range(len(par_files)):
    par_file = par_files[id]
    bx, by, bz, phi, metrics, fig = au.disambig_azimuth(bhor, blos, par_file, ambig_executable_path, id, plot_fig=True, save_dir=save_dir, save_fig=True, data_mask=data_mask)
    # save the disambiguated magnetic field components to a npz file
    ambig_outfile = os.path.join(save_dir, f'ambig_out_{id}.npz')
    # remove the ambig_outfile if it already exists
    if os.path.exists(ambig_outfile):
        os.remove(ambig_outfile)
    np.savez(ambig_outfile, bx=bx, by=by, bz=bz, phi=phi, metrics=metrics)
    ambig_outfiles.append(ambig_outfile)
    ambig_metrics.append(metrics)
    ambig_figs.append(fig)
    clear_output()

In [ ]:

# Display each figure in the list
for fig in ambig_figs:
    display(fig)
    plt.close(fig)  # Close the figure after displaying to avoid re-displaying in later cells


In [ ]:
keys = list(ambig_metrics[0].keys())
count = 0
for metric in ambig_metrics:
    print(f'{count}: {id_dict[count][0]}: {id_dict[count][1]}:', end=' ')
    for key in keys:
        print(f'{metric[key]:.2f}', end=' ')
    print()
    count += 1